In [ ]:
!pip install q scipy==1.1.0  #use this version of scipy for proper working of sparsematrix operations

     |████████████████████████████████| 31.2MB 108kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
import torch.nn.functional as F
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats
from torch.autograd import Variable

In [ ]:
cuda = torch.device('cuda')     # Default CUDA device

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
#define CNN source = https://zhenye-na.github.io/2018/09/28/pytorch-cnn-cifar10.html
#padding same 
#CNN architecture using 3 blocks of VGGnet for desired accuracy 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5,padding = (2,2))
        self.conv2 = nn.Conv2d(32, 32, 5,padding = (2,2))
        self.conv3 = nn.Conv2d(32, 64, 5,padding = (2,2))
        self.conv4 = nn.Conv2d(64, 64, 5,padding = (2,2))
        self.conv5 = nn.Conv2d(64, 128, 5,padding = (2,2))
        self.conv6 = nn.Conv2d(128, 128, 5,padding = (2,2))
        self.pool = nn.MaxPool2d(2, 2)
        # self.fc1 = nn.Linear(int(32768/64), 10)
        self.fc1 = nn.Linear(128*4*4,10)
        
        self.drpout = nn.Dropout(0.2)
        # self.out = nn.Softmax(dim = 2) #hidhaval

    def forward(self, x):
        x = (F.relu(self.conv1(x)))                           #(3,32,32) -> (32,28,28)
        x = self.drpout(self.pool(F.relu(self.conv2(x))))     #(32,28,28)-> (32,12,12)
        x = (F.relu(self.conv3(x)))                           #(32,12,12)->(64,8,8)
        x = self.drpout(self.pool(F.relu(self.conv4(x))))     #(64,8,8) ->(64,2,2)
        x = (F.relu(self.conv5(x)))
        x = self.drpout(self.pool(F.relu(self.conv6(x))))
        x = x.view(-1,128*4*4)
        # # x = F.relu(self.fc1(x))
        x = self.fc1(x)
        # # x = self.out(x) #hidhaval
        return x
#set random seed for reproducibility
torch.manual_seed(0)

net = Net()
net = net.to(cuda)
torch.save(net.state_dict(),'weights_i.pth')

In [ ]:
# wi1 = wi1.ravel().squeeze()


In [ ]:
#training
def one_hot_embedding(labels, num_classes):
    """Embedding labels to one-hot form.

    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.

    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes) 
    return y[labels] 



criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001, weight_decay=0.001)
for epoch in range(40):  # loop over the dataset multiple times
    correct = 0
    total = 0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(cuda)
        # labelsOH = one_hot_embedding(labels,10).long()
        # labelsOH = labelsOH.to(cuda)
        labels = labels.to(cuda)
        # zero the parameter gradients
        optimizer.zero_grad()

#         # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()        

        # print statistics
        running_loss += loss.item()
    print("Epoch Number: "+str(epoch+1)+'Training Accuracy: %d %%' % (
    100 * correct / total))

    # print("Epoch Number: "+str(epoch+1)+" Loss is: "+str(running_loss / 60000))
    # print('[%d] loss: %.3f' %(epoch + 1), (running_loss / 60000))

#save learned weights
torch.save(net.state_dict(),'weights_f.pth')


Epoch Number: 1Training Accuracy: 33 %
Epoch Number: 2Training Accuracy: 46 %
Epoch Number: 3Training Accuracy: 52 %
Epoch Number: 4Training Accuracy: 56 %
Epoch Number: 5Training Accuracy: 59 %
Epoch Number: 6Training Accuracy: 62 %
Epoch Number: 7Training Accuracy: 65 %
Epoch Number: 8Training Accuracy: 67 %
Epoch Number: 9Training Accuracy: 69 %
Epoch Number: 10Training Accuracy: 70 %
Epoch Number: 11Training Accuracy: 71 %
Epoch Number: 12Training Accuracy: 73 %
Epoch Number: 13Training Accuracy: 74 %
Epoch Number: 14Training Accuracy: 75 %
Epoch Number: 15Training Accuracy: 76 %
Epoch Number: 16Training Accuracy: 77 %
Epoch Number: 17Training Accuracy: 78 %
Epoch Number: 18Training Accuracy: 79 %
Epoch Number: 19Training Accuracy: 80 %
Epoch Number: 20Training Accuracy: 81 %
Epoch Number: 21Training Accuracy: 82 %
Epoch Number: 22Training Accuracy: 83 %
Epoch Number: 23Training Accuracy: 83 %
Epoch Number: 24Training Accuracy: 84 %
Epoch Number: 25Training Accuracy: 85 %
Epoch Num

In [ ]:
#testing #40 epochs
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(cuda)
        labels = labels.to(cuda)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
acc = (correct/total)
np.save('model_accuracy_3.npy',acc)

Accuracy of the network on the 10000 test images: 76 %


In [ ]:
wi =torch.load('weights_i.pth')
wi.keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'conv3.weight', 'conv3.bias', 'conv4.weight', 'conv4.bias', 'conv5.weight', 'conv5.bias', 'conv6.weight', 'conv6.bias', 'fc1.weight', 'fc1.bias'])

In [ ]:
wi['fc1.weight'].size()

torch.Size([10, 2048])

In [ ]:
wc1 = wi['conv1.weight']
wc1[(0,0,0)]

tensor([-0.0039,  0.0381, -0.0864,  0.0335, -0.0676], device='cuda:0')

In [ ]:
wc1flat = wc1.resize((32*3*5*5))

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [ ]:
wc1flat.resize(32,3,5,5)[(0,0,0)]

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


tensor([-0.0039,  0.0381, -0.0864,  0.0335, -0.0676], device='cuda:0')

In [ ]:
#vectorize wi
wi =torch.load('weights_i.pth')

wi1 = np.asarray(wi['conv1.weight'].cpu().resize(1,32*3*5*5))
wi2 = np.asarray(wi['conv2.weight'].cpu().resize(1,32*32*5*5))
wi3 = np.asarray(wi['conv3.weight'].cpu().resize(1,64*32*5*5))
wi4 = np.asarray(wi['conv4.weight'].cpu().resize(1,64*64*5*5))
wi5 = np.asarray(wi['conv5.weight'].cpu().resize(1,128*64*5*5))
wi6 = np.asarray(wi['conv6.weight'].cpu().resize(1,128*128*5*5))
wi7 = np.asarray(wi['fc1.weight'].cpu().resize(1,10*2048))  #size of the resulting image after the given layers of convolutions 

wi_vec = np.concatenate((wi1,wi2,wi3,wi4,wi5,wi6,wi7),axis =1)
wi_vec = wi_vec.T
dim_w = wi_vec.shape[0]
wi_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([-0.00086451,  0.06194317, -0.09503707, ..., -0.01400343,
        0.00669239, -0.0133841 ], dtype=float32)

In [ ]:
dim_w

816480

In [ ]:
#vectorize wf
wf  =torch.load('weights_f.pth')

wf1 = np.asarray(wf['conv1.weight'].cpu().resize(1,32*3*5*5))
wf2 = np.asarray(wf['conv2.weight'].cpu().resize(1,32*32*5*5))
wf3 = np.asarray(wf['conv3.weight'].cpu().resize(1,64*32*5*5))
wf4 = np.asarray(wf['conv4.weight'].cpu().resize(1,64*64*5*5))              
wf5 = np.asarray(wf['conv5.weight'].cpu().resize(1,128*64*5*5))
wf6 = np.asarray(wf['conv6.weight'].cpu().resize(1,128*128*5*5))
wf7 = np.asarray(wf['fc1.weight'].cpu().resize(1,10*2048))  #size of the resulting image after the given layers of convolutions 

wf_vec = np.concatenate((wf1,wf2,wf3,wf4,wf5,wf6,wf7),axis =1)
wf_vec = wf_vec.T
wf_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([ 0.01560975,  0.1377461 , -0.10281772, ...,  0.00143891,
        0.01559104,  0.00204668], dtype=float32)

In [ ]:
#Sparse Matrix A used for projection
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w        #density of sparse matrix
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228])   #size of projection
n_runs = 2            # no. of times the experiment is run to account for random initializations
acc  = np.zeros((n_runs,num_rows_arr.shape[0])) #store accuracy
dist_i = np.zeros((n_runs,num_rows_arr.shape[0])) #store distance from wi
dist_f = np.zeros((n_runs,num_rows_arr.shape[0])) #store distance from wf

for i_out in range(n_runs):   #outer loop iterating over random initializations
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):  #inner loop iterating over increasing dimensions of A vector
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      #rearrange vectorized weight vector to feed to the network for testing
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1

 Dist from Wi = 0.012782785756203877 , Dist from Wf =18.58079004787988

 n_rows = 4 , Accuracy =0.1

 Dist from Wi = 0.0258706840340926 , Dist from Wf =18.580776434549392

 n_rows = 8 , Accuracy =0.1

 Dist from Wi = 0.03492448569112732 , Dist from Wf =18.58076162279908

 n_rows = 16 , Accuracy =0.1

 Dist from Wi = 0.054755831621565124 , Dist from Wf =18.58071376459585

 n_rows = 32 , Accuracy =0.1

 Dist from Wi = 0.11005554422436872 , Dist from Wf =18.580468508093595

 n_rows = 64 , Accuracy =0.1

 Dist from Wi = 0.17212159770449742 , Dist from Wf =18.579997210944885

 n_rows = 128 , Accuracy =0.1

 Dist from Wi = 0.234214102641464 , Dist from Wf =18.579318231765537

 n_rows = 256 , Accuracy =0.1

 Dist from Wi = 0.3641267893300866 , Dist from Wf =18.577226216114358

 n_rows = 512 , Accuracy =0.1

 Dist from Wi = 0.49216370193350817 , Dist from Wf =18.574275143148096

 n_rows = 1024 , Accuracy =0.1

 Dist from Wi = 0.6725041393

In [ ]:
#save accuracy and distances from initial and  final weight vectors
np.save("accuracy_A.npy", acc)
np.save("distance_i_A.npy", dist_i)
np.save("distance_f_A.npy", dist_f)

In [ ]:
#same process but for A+I instead of the earlier sparse A matrix
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
I = sparse.identity(dim_w)

num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1

 Dist from Wi = 0.049525077929838006 , Dist from Wf =18.580728442923984

 n_rows = 4 , Accuracy =0.1

 Dist from Wi = 0.06409737857419219 , Dist from Wf =18.580683887552805

 n_rows = 8 , Accuracy =0.1

 Dist from Wi = 0.07598756112343726 , Dist from Wf =18.580639065802448

 n_rows = 16 , Accuracy =0.1

 Dist from Wi = 0.08650803712354874 , Dist from Wf =18.580593062725995

 n_rows = 32 , Accuracy =0.1

 Dist from Wi = 0.12171438890835658 , Dist from Wf =18.580395792624234

 n_rows = 64 , Accuracy =0.1

 Dist from Wi = 0.16450798944562073 , Dist from Wf =18.580066181918333

 n_rows = 128 , Accuracy =0.1

 Dist from Wi = 0.2327254104075993 , Dist from Wf =18.579336938825783

 n_rows = 256 , Accuracy =0.1

 Dist from Wi = 0.3316345913516895 , Dist from Wf =18.577834661250076

 n_rows = 512 , Accuracy =0.1

 Dist from Wi = 0.4822143970103413 , Dist from Wf =18.574536103984276

 n_rows = 1024 , Accuracy =0.1

 Dist from Wi = 0.6905607

In [ ]:
np.save("accuracy_AI.npy", acc)
np.save("distance_i_AI.npy", dist_i)
np.save("distance_f_AI.npy", dist_f)

In [ ]:
# Projecting along I matrix 
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228,32*3*25,32*3*25+32*32*25,32*3*25+32*32*25+64*32*25,32*3*25+32*32*25+64*32*25+64*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25+20480])
num_rows_arr = np.sort(num_rows_arr)
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      #Initial n_rows number of rows go to wf and remaining go to wi
      w = np.zeros((dim_w,1))
      w[0:n_rows] = wf_vec[0:n_rows]
      w[n_rows:] = wi_vec[n_rows:]

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1

 Dist from Wi = 0.07757244407949496 , Dist from Wf =18.580632516653225

 n_rows = 4 , Accuracy =0.1

 Dist from Wi = 0.07854549092166721 , Dist from Wf =18.580628428792377

 n_rows = 8 , Accuracy =0.1

 Dist from Wi = 0.13756590397431964 , Dist from Wf =18.580285192242215

 n_rows = 16 , Accuracy =0.1

 Dist from Wi = 0.14976807968608247 , Dist from Wf =18.580190842004257

 n_rows = 32 , Accuracy =0.1

 Dist from Wi = 0.17966930180687155 , Dist from Wf =18.57992575725156

 n_rows = 64 , Accuracy =0.1

 Dist from Wi = 0.22198954043799413 , Dist from Wf =18.579468314430525

 n_rows = 128 , Accuracy =0.1

 Dist from Wi = 0.30970686694281774 , Dist from Wf =18.578213150342528

 n_rows = 256 , Accuracy =0.1

 Dist from Wi = 0.4068163823528191 , Dist from Wf =18.576340399390862

 n_rows = 512 , Accuracy =0.1

 Dist from Wi = 0.6019747506311249 , Dist from Wf =18.571040590192954

 n_rows = 1024 , Accuracy =0.1

 Dist from Wi = 0.84336373

In [ ]:
np.save("accuracy_I.npy", acc)
np.save("distance_i_I.npy", dist_i)
np.save("distance_f_I.npy", dist_f)

In [ ]:
# Irev
# Same operation as above on secondary Identity matrix
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228,32*3*25,32*3*25+32*32*25,32*3*25+32*32*25+64*32*25,32*3*25+32*32*25+64*32*25+64*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25+20480])
num_rows_arr = np.sort(num_rows_arr)
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      w = np.zeros((dim_w,1))
      w[0:-n_rows] = wi_vec[0:-n_rows]
      w[-n_rows:] = wf_vec[-n_rows:] 
      
      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1

 Dist from Wi = 0.01781276788960192 , Dist from Wf =18.58078590663731

 n_rows = 4 , Accuracy =0.1

 Dist from Wi = 0.024003418667525894 , Dist from Wf =18.580778940584917

 n_rows = 8 , Accuracy =0.1

 Dist from Wi = 0.047257904473935575 , Dist from Wf =18.580734347529333

 n_rows = 16 , Accuracy =0.1

 Dist from Wi = 0.1017199439964933 , Dist from Wf =18.58051601156397

 n_rows = 32 , Accuracy =0.1

 Dist from Wi = 0.12776250546219223 , Dist from Wf =18.58035518888672

 n_rows = 64 , Accuracy =0.1

 Dist from Wi = 0.1870877270188688 , Dist from Wf =18.579852539387673

 n_rows = 128 , Accuracy =0.1

 Dist from Wi = 0.2825887108041211 , Dist from Wf =18.578645424882776

 n_rows = 256 , Accuracy =0.1

 Dist from Wi = 0.4205256231607485 , Dist from Wf =18.576035109873605

 n_rows = 512 , Accuracy =0.1

 Dist from Wi = 0.5980233029268971 , Dist from Wf =18.571168254370754

 n_rows = 1024 , Accuracy =0.1

 Dist from Wi = 0.88534849118

In [ ]:
np.save("accuracy_UI.npy", acc)
np.save("distance_i_UI.npy", dist_i)
np.save("distance_f_UI.npy", dist_f)

In [ ]:
from google.colab import files
files.download('accuracy_UI.npy') 
files.download('distance_i_UI.npy')
files.download('distance_f_UI.npy')

files.download('accuracy_I.npy') 
files.download('distance_i_I.npy')
files.download('distance_f_I.npy')

files.download('accuracy_AI.npy') 
files.download('distance_i_AI.npy')
files.download('distance_f_AI.npy')

files.download('accuracy_A.npy') 
files.download('distance_i_A.npy')
files.download('distance_f_A.npy')

In [ ]:
# Following blocks are just subsequent runs of the above experiments to rule out the effect of random initialization
# A run 2
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.0979

 Dist from Wi = 0.017016106258059657 , Dist from Wf =18.869534801508372

 n_rows = 4 , Accuracy =0.1014

 Dist from Wi = 0.017211490089415117 , Dist from Wf =18.869534624304237

 n_rows = 8 , Accuracy =0.0984

 Dist from Wi = 0.03439665074068998 , Dist from Wf =18.86951112359879

 n_rows = 16 , Accuracy =0.1024

 Dist from Wi = 0.07661009268966612 , Dist from Wf =18.869386955250228

 n_rows = 32 , Accuracy =0.0995

 Dist from Wi = 0.1233910558703518 , Dist from Wf =18.86913903231209

 n_rows = 64 , Accuracy =0.1003

 Dist from Wi = 0.1664620384409039 , Dist from Wf =18.86880821788096

 n_rows = 128 , Accuracy =0.0999

 Dist from Wi = 0.2337763545457206 , Dist from Wf =18.868094280808982

 n_rows = 256 , Accuracy =0.1001

 Dist from Wi = 0.331707735164718 , Dist from Wf =18.866626703036598

 n_rows = 512 , Accuracy =0.1005

 Dist from Wi = 0.4807989984964341 , Dist from Wf =18.86341606112603

 n_rows = 1024 , Accuracy =0.0997

 

In [ ]:
# A run 3
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1

 Dist from Wi = 0.027450839283004527 , Dist from Wf =18.639100077652657

 n_rows = 4 , Accuracy =0.1

 Dist from Wi = 0.03114020335186143 , Dist from Wf =18.639094278989745

 n_rows = 8 , Accuracy =0.1

 Dist from Wi = 0.050631030215054676 , Dist from Wf =18.639051525012047

 n_rows = 16 , Accuracy =0.1001

 Dist from Wi = 0.09021655232706603 , Dist from Wf =18.63890195872596

 n_rows = 32 , Accuracy =0.1

 Dist from Wi = 0.12825782163631766 , Dist from Wf =18.6386790085696

 n_rows = 64 , Accuracy =0.1001

 Dist from Wi = 0.19110768173278375 , Dist from Wf =18.638140548540758

 n_rows = 128 , Accuracy =0.1

 Dist from Wi = 0.25863399813598725 , Dist from Wf =18.637325819668323

 n_rows = 256 , Accuracy =0.1001

 Dist from Wi = 0.3360202863619583 , Dist from Wf =18.636091210892612

 n_rows = 512 , Accuracy =0.1

 Dist from Wi = 0.45293345921537037 , Dist from Wf =18.633616303202704

 n_rows = 1024 , Accuracy =0.1

 Dist from Wi = 

In [ ]:
np.save("accuracy_3.npy", acc)
np.save("distance_i_3.npy", dist_i)
np.save("distance_f_3.npy", dist_f)

In [ ]:
#initialize A+I
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
I = sparse.identity(dim_w)

num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1

 Dist from Wi = 0.0413549524091219 , Dist from Wf =18.613316295795123

 n_rows = 4 , Accuracy =0.1

 Dist from Wi = 0.054795559092620585 , Dist from Wf =18.6132815807982

 n_rows = 8 , Accuracy =0.1

 Dist from Wi = 0.0605899228042822 , Dist from Wf =18.613263620886116

 n_rows = 16 , Accuracy =0.1

 Dist from Wi = 0.08372574259976674 , Dist from Wf =18.613173930295485

 n_rows = 32 , Accuracy =0.1

 Dist from Wi = 0.13113415447594337 , Dist from Wf =18.612900300411688

 n_rows = 64 , Accuracy =0.1

 Dist from Wi = 0.17170657330834682 , Dist from Wf =18.61257023121329

 n_rows = 128 , Accuracy =0.1

 Dist from Wi = 0.24186041542510311 , Dist from Wf =18.611790813853904

 n_rows = 256 , Accuracy =0.1

 Dist from Wi = 0.3519166545503112 , Dist from Wf =18.610035153849605

 n_rows = 512 , Accuracy =0.1

 Dist from Wi = 0.5002956101570359 , Dist from Wf =18.606637473266

 n_rows = 1024 , Accuracy =0.1

 Dist from Wi = 0.69382623517338

In [ ]:
# A+I run 2 
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
I = sparse.identity(dim_w)

num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1005

 Dist from Wi = 0.025627126791223583 , Dist from Wf =18.869525071489836

 n_rows = 4 , Accuracy =0.1006

 Dist from Wi = 0.04051405823537975 , Dist from Wf =18.869498980742108

 n_rows = 8 , Accuracy =0.1

 Dist from Wi = 0.05833046931831902 , Dist from Wf =18.869452316628685

 n_rows = 16 , Accuracy =0.0999

 Dist from Wi = 0.08280787110405001 , Dist from Wf =18.869360774273083

 n_rows = 32 , Accuracy =0.1005

 Dist from Wi = 0.10419091095438617 , Dist from Wf =18.869254819078918

 n_rows = 64 , Accuracy =0.0979

 Dist from Wi = 0.1647375516780402 , Dist from Wf =18.868823352616808

 n_rows = 128 , Accuracy =0.0975

 Dist from Wi = 0.23110742004634052 , Dist from Wf =18.868127160199368

 n_rows = 256 , Accuracy =0.0969

 Dist from Wi = 0.3195005189249446 , Dist from Wf =18.86683737651035

 n_rows = 512 , Accuracy =0.1005

 Dist from Wi = 0.4668410907685896 , Dist from Wf =18.863766659102982

 n_rows = 1024 , Accuracy =0.1011


In [ ]:
#initialize A+I run 3
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
I = sparse.identity(dim_w)

num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1001

 Dist from Wi = 0.026179942700650237 , Dist from Wf =18.639101906045816

 n_rows = 4 , Accuracy =0.1

 Dist from Wi = 0.027631654721269213 , Dist from Wf =18.639099810479347

 n_rows = 8 , Accuracy =0.1

 Dist from Wi = 0.03512127418361911 , Dist from Wf =18.639087202689918

 n_rows = 16 , Accuracy =0.1

 Dist from Wi = 0.06654018083462085 , Dist from Wf =18.63900151986941

 n_rows = 32 , Accuracy =0.1002

 Dist from Wi = 0.10170146547789657 , Dist from Wf =18.63884283063914

 n_rows = 64 , Accuracy =0.1

 Dist from Wi = 0.13998035539240714 , Dist from Wf =18.638594656073693

 n_rows = 128 , Accuracy =0.1002

 Dist from Wi = 0.2126636321008383 , Dist from Wf =18.63790705612176

 n_rows = 256 , Accuracy =0.1

 Dist from Wi = 0.33919012976754886 , Dist from Wf =18.636033786975034

 n_rows = 512 , Accuracy =0.1

 Dist from Wi = 0.4704600067103593 , Dist from Wf =18.6331820319501

 n_rows = 1024 , Accuracy =0.0999

 Dist from Wi = 

In [ ]:
np.save("accuracy_AI3.npy", acc)
np.save("distance_i_AI3.npy", dist_i)
np.save("distance_f_AI3.npy", dist_f)

In [ ]:
# I
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
I = sparse.identity(dim_w)

num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228])
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1

 Dist from Wi = 0.014579535432896916 , Dist from Wf =18.61335652687477

 n_rows = 4 , Accuracy =0.1

 Dist from Wi = 0.015612711920947875 , Dist from Wf =18.613355688930234

 n_rows = 8 , Accuracy =0.1

 Dist from Wi = 0.03043628304619056 , Dist from Wf =18.613337352341716

 n_rows = 16 , Accuracy =0.1

 Dist from Wi = 0.07151013024988881 , Dist from Wf =18.61322486998193

 n_rows = 32 , Accuracy =0.1

 Dist from Wi = 0.0831266905624824 , Dist from Wf =18.613176615304642

 n_rows = 64 , Accuracy =0.1

 Dist from Wi = 0.11236330992172044 , Dist from Wf =18.61302308185833

 n_rows = 128 , Accuracy =0.1

 Dist from Wi = 0.21419534675851473 , Dist from Wf =18.612129757575357

 n_rows = 256 , Accuracy =0.1

 Dist from Wi = 0.34554921032746627 , Dist from Wf =18.610154472831937

 n_rows = 512 , Accuracy =0.1

 Dist from Wi = 0.5693276487545821 , Dist from Wf =18.604653175665387

 n_rows = 1024 , Accuracy =0.1

 Dist from Wi = 0.86094657

In [ ]:
# I run 2 
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
I = sparse.identity(dim_w)

num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228])
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1018

 Dist from Wi = 0.03447624379238255 , Dist from Wf =18.869510978343975

 n_rows = 4 , Accuracy =0.0999

 Dist from Wi = 0.05033948529980653 , Dist from Wf =18.86947532681949

 n_rows = 8 , Accuracy =0.0996

 Dist from Wi = 0.07616984420944803 , Dist from Wf =18.869388737531033

 n_rows = 16 , Accuracy =0.0999

 Dist from Wi = 0.13244833642755227 , Dist from Wf =18.869077630121467

 n_rows = 32 , Accuracy =0.1002

 Dist from Wi = 0.17367921147102458 , Dist from Wf =18.868743167071802

 n_rows = 64 , Accuracy =0.1001

 Dist from Wi = 0.24037623863857158 , Dist from Wf =18.86801135353999

 n_rows = 128 , Accuracy =0.098

 Dist from Wi = 0.3311136230529674 , Dist from Wf =18.86663713920308

 n_rows = 256 , Accuracy =0.101

 Dist from Wi = 0.407368090657988 , Dist from Wf =18.86514469629787

 n_rows = 512 , Accuracy =0.0984

 Dist from Wi = 0.539928099499993 , Dist from Wf =18.861816212153883

 n_rows = 1024 , Accuracy =0.1004

 Dis

In [ ]:
# I run 3
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
I = sparse.identity(dim_w)

num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228])
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1002

 Dist from Wi = 0.03413085790772091 , Dist from Wf =18.639089042598744

 n_rows = 4 , Accuracy =0.1001

 Dist from Wi = 0.038699563916374725 , Dist from Wf =18.63908011671013

 n_rows = 8 , Accuracy =0.1

 Dist from Wi = 0.04839546390345393 , Dist from Wf =18.639057463630156

 n_rows = 16 , Accuracy =0.1

 Dist from Wi = 0.08387839069141427 , Dist from Wf =18.63893155921252

 n_rows = 32 , Accuracy =0.1

 Dist from Wi = 0.11990409994367933 , Dist from Wf =18.638734620683085

 n_rows = 64 , Accuracy =0.1

 Dist from Wi = 0.16273952879410516 , Dist from Wf =18.638409832901367

 n_rows = 128 , Accuracy =0.1

 Dist from Wi = 0.21147559753787862 , Dist from Wf =18.637920574056658

 n_rows = 256 , Accuracy =0.1

 Dist from Wi = 0.3938798399958853 , Dist from Wf =18.634958114404775

 n_rows = 512 , Accuracy =0.0999

 Dist from Wi = 0.5618996635928705 , Dist from Wf =18.63064878156957

 n_rows = 1024 , Accuracy =0.1

 Dist from Wi = 0.

In [ ]:
np.save("accuracy_I3.npy", acc)
np.save("distance_i_I3.npy", dist_i)
np.save("distance_f_I3.npy", dist_f)

In [ ]:
#I UI experiment


!pip install q scipy==1.1.0
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
import torch.nn.functional as F
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats
from torch.autograd import Variable

     |████████████████████████████████| 31.2MB 105kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
cuda = torch.device('cuda')     # Default CUDA device
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#define CNN source = https://zhenye-na.github.io/2018/09/28/pytorch-cnn-cifar10.html
#padding same 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5,padding = (2,2))
        self.conv2 = nn.Conv2d(32, 32, 5,padding = (2,2))
        self.conv3 = nn.Conv2d(32, 64, 5,padding = (2,2))
        self.conv4 = nn.Conv2d(64, 64, 5,padding = (2,2))
        self.conv5 = nn.Conv2d(64, 128, 5,padding = (2,2))
        self.conv6 = nn.Conv2d(128, 128, 5,padding = (2,2))
        self.pool = nn.MaxPool2d(2, 2)
        # self.fc1 = nn.Linear(int(32768/64), 10)
        self.fc1 = nn.Linear(128*4*4,10)
        
        self.drpout = nn.Dropout(0.2)
        # self.out = nn.Softmax(dim = 2) #hidhaval

    def forward(self, x):
        x = (F.relu(self.conv1(x)))                           #(3,32,32) -> (32,28,28)
        x = self.drpout(self.pool(F.relu(self.conv2(x))))     #(32,28,28)-> (32,12,12)
        x = (F.relu(self.conv3(x)))                           #(32,12,12)->(64,8,8)
        x = self.drpout(self.pool(F.relu(self.conv4(x))))     #(64,8,8) ->(64,2,2)
        x = (F.relu(self.conv5(x)))
        x = self.drpout(self.pool(F.relu(self.conv6(x))))
        x = x.view(-1,128*4*4)
        # # x = F.relu(self.fc1(x))
        x = self.fc1(x)
        # # x = self.out(x) #hidhaval
        return x

net = Net()
net = net.to(cuda)
torch.save(net.state_dict(),'weights_i.pth')

In [ ]:
#training
#hiumang
def one_hot_embedding(labels, num_classes):
    """Embedding labels to one-hot form.

    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.

    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes) 
    return y[labels] 



criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001, weight_decay=0.001)
for epoch in range(40):  # loop over the dataset multiple times
    correct = 0
    total = 0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(cuda)
        # labelsOH = one_hot_embedding(labels,10).long()
        # labelsOH = labelsOH.to(cuda)
        labels = labels.to(cuda)
        # zero the parameter gradients
        optimizer.zero_grad()

#         # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()        

        # print statistics
        running_loss += loss.item()
    print("Epoch Number: "+str(epoch+1)+'Training Accuracy: %d %%' % (
    100 * correct / total))

    # print("Epoch Number: "+str(epoch+1)+" Loss is: "+str(running_loss / 60000))
    # print('[%d] loss: %.3f' %(epoch + 1), (running_loss / 60000))

#save learned weights
torch.save(net.state_dict(),'weights_f.pth')


Epoch Number: 1Training Accuracy: 34 %
Epoch Number: 2Training Accuracy: 46 %
Epoch Number: 3Training Accuracy: 52 %
Epoch Number: 4Training Accuracy: 56 %
Epoch Number: 5Training Accuracy: 59 %
Epoch Number: 6Training Accuracy: 62 %
Epoch Number: 7Training Accuracy: 64 %
Epoch Number: 8Training Accuracy: 66 %
Epoch Number: 9Training Accuracy: 68 %
Epoch Number: 10Training Accuracy: 70 %
Epoch Number: 11Training Accuracy: 71 %
Epoch Number: 12Training Accuracy: 73 %
Epoch Number: 13Training Accuracy: 74 %
Epoch Number: 14Training Accuracy: 75 %
Epoch Number: 15Training Accuracy: 76 %
Epoch Number: 16Training Accuracy: 78 %
Epoch Number: 17Training Accuracy: 79 %
Epoch Number: 18Training Accuracy: 80 %
Epoch Number: 19Training Accuracy: 81 %
Epoch Number: 20Training Accuracy: 82 %
Epoch Number: 21Training Accuracy: 83 %
Epoch Number: 22Training Accuracy: 83 %
Epoch Number: 23Training Accuracy: 85 %
Epoch Number: 24Training Accuracy: 85 %
Epoch Number: 25Training Accuracy: 87 %
Epoch Num

In [ ]:
#testing #40 epochs
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(cuda)
        labels = labels.to(cuda)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
acc = (correct/total)
np.save('model_accuracy_3.npy',acc)

Accuracy of the network on the 10000 test images: 73 %


In [ ]:
#vectorize wi
wi =torch.load('weights_i.pth')

wi1 = np.asarray(wi['conv1.weight'].cpu().resize(1,32*3*5*5))
wi2 = np.asarray(wi['conv2.weight'].cpu().resize(1,32*32*5*5))
wi3 = np.asarray(wi['conv3.weight'].cpu().resize(1,64*32*5*5))
wi4 = np.asarray(wi['conv4.weight'].cpu().resize(1,64*64*5*5))
wi5 = np.asarray(wi['conv5.weight'].cpu().resize(1,128*64*5*5))
wi6 = np.asarray(wi['conv6.weight'].cpu().resize(1,128*128*5*5))
wi7 = np.asarray(wi['fc1.weight'].cpu().resize(1,10*2048))  #size of the resulting image after the given layers of convolutions 

wi_vec = np.concatenate((wi1,wi2,wi3,wi4,wi5,wi6,wi7),axis =1)
wi_vec = wi_vec.T
dim_w = wi_vec.shape[0]
wi_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([-0.04525725, -0.07346912, -0.0181139 , ...,  0.00749681,
       -0.00077182,  0.01913429], dtype=float32)

In [ ]:
#vectorize wf
wf  =torch.load('weights_f.pth')

wf1 = np.asarray(wf['conv1.weight'].cpu().resize(1,32*3*5*5))
wf2 = np.asarray(wf['conv2.weight'].cpu().resize(1,32*32*5*5))
wf3 = np.asarray(wf['conv3.weight'].cpu().resize(1,64*32*5*5))
wf4 = np.asarray(wf['conv4.weight'].cpu().resize(1,64*64*5*5))              
wf5 = np.asarray(wf['conv5.weight'].cpu().resize(1,128*64*5*5))
wf6 = np.asarray(wf['conv6.weight'].cpu().resize(1,128*128*5*5))
wf7 = np.asarray(wf['fc1.weight'].cpu().resize(1,10*2048))  #size of the resulting image after the given layers of convolutions 

wf_vec = np.concatenate((wf1,wf2,wf3,wf4,wf5,wf6,wf7),axis =1)
wf_vec = wf_vec.T
wf_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([-0.04987422, -0.09904844, -0.04742719, ...,  0.02819859,
        0.05579428,  0.02368088], dtype=float32)

In [ ]:
# I
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228,32*3*25,32*3*25+32*32*25,32*3*25+32*32*25+64*32*25,32*3*25+32*32*25+64*32*25+64*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25+20480])
num_rows_arr = np.sort(num_rows_arr)
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      w = np.zeros((dim_w,1))
      w[0:n_rows] = wf_vec[0:n_rows]
      w[n_rows:] = wi_vec[n_rows:]

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1

 Dist from Wi = 0.025992660051979248 , Dist from Wf =18.644531418434795

 n_rows = 4 , Accuracy =0.1

 Dist from Wi = 0.03971569829233779 , Dist from Wf =18.644507236574743

 n_rows = 8 , Accuracy =0.1

 Dist from Wi = 0.050286744154282255 , Dist from Wf =18.644481721805544

 n_rows = 16 , Accuracy =0.1

 Dist from Wi = 0.0641695570940345 , Dist from Wf =18.64443910927119

 n_rows = 32 , Accuracy =0.1

 Dist from Wi = 0.08731940045632276 , Dist from Wf =18.644345061001275

 n_rows = 64 , Accuracy =0.1

 Dist from Wi = 0.17518640425608364 , Dist from Wf =18.643726482523352

 n_rows = 128 , Accuracy =0.1

 Dist from Wi = 0.28033585819015794 , Dist from Wf =18.642441879699977

 n_rows = 256 , Accuracy =0.1

 Dist from Wi = 0.4174476052048205 , Dist from Wf =18.639875668262583

 n_rows = 512 , Accuracy =0.1

 Dist from Wi = 0.5559864894914488 , Dist from Wf =18.63625784471984

 n_rows = 1024 , Accuracy =0.1

 Dist from Wi = 0.84665815

In [ ]:
np.save("accuracy_I3.npy", acc)
np.save("distance_i_I3.npy", dist_i)
np.save("distance_f_I3.npy", dist_f)

In [ ]:
# Irev
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,262144,524228,32*3*25,32*3*25+32*32*25,32*3*25+32*32*25+64*32*25,32*3*25+32*32*25+64*32*25+64*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25,32*3*25+32*32*25+64*32*25+64*64*25+128*64*25+128*128*25+20480])
num_rows_arr = np.sort(num_rows_arr)
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      w = np.zeros((dim_w,1))
      w[0:-n_rows] = wi_vec[0:-n_rows]
      w[-n_rows:] = wf_vec[-n_rows:]

      #test performance of this reinitialization
      w1= w[0:32*3*5*5]
      w1 = torch.from_numpy(w1).reshape(32,3,5,5)
      w2 = w[32*3*5*5:32*3*5*5+32*32*5*5]
      w2 = torch.from_numpy(w2).reshape(32,32,5,5)
      w3 = w[32*3*5*5+32*32*5*5:32*3*5*5+32*32*5*5+ 64*32*5*5]
      w3 = torch.from_numpy(w3).reshape(64,32,5,5)
      w4 = w[32*3*5*5+32*32*5*5+ 64*32*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5]
      w4 = torch.from_numpy(w4).reshape(64,64,5,5)
      w5 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5: 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5+ 128*64*5*5]
      w5 = torch.from_numpy(w5).reshape(128,64,5,5)
      w6 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 :32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5]
      w6 = torch.from_numpy(w6).reshape(128,128,5,5)
      w7 = w[32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 : 32*3*5*5+32*32*5*5+ 64*32*5*5 + 64*64*5*5 + 128*64*5*5 + 128*128*5*5 + 2048*10]
      w7 = torch.from_numpy(w7).reshape(10,2048)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['conv3.weight'] = (w3)
      wf['conv4.weight'] = (w4)
      wf['conv5.weight'] = (w5)
      wf['conv6.weight'] = (w6)
      wf['fc1.weight'] = (w7)
      # wf.to(cuda)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images = images.to(cuda)
              labels = labels.to(cuda)
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1

 n_rows = 2 , Accuracy =0.1

 Dist from Wi = 0.0567485278152299 , Dist from Wf =18.64446317371379

 n_rows = 4 , Accuracy =0.1

 Dist from Wi = 0.06156432014257908 , Dist from Wf =18.644447893833277

 n_rows = 8 , Accuracy =0.1

 Dist from Wi = 0.08138169928137899 , Dist from Wf =18.64437192426714

 n_rows = 16 , Accuracy =0.1

 Dist from Wi = 0.11017146649745535 , Dist from Wf =18.644224029960284

 n_rows = 32 , Accuracy =0.1

 Dist from Wi = 0.1771423419741833 , Dist from Wf =18.64370800087951

 n_rows = 64 , Accuracy =0.1

 Dist from Wi = 0.20524389997746978 , Dist from Wf =18.643419814317888

 n_rows = 128 , Accuracy =0.1

 Dist from Wi = 0.31963389130865727 , Dist from Wf =18.64180950462979

 n_rows = 256 , Accuracy =0.1

 Dist from Wi = 0.3976370311940684 , Dist from Wf =18.64030880170181

 n_rows = 512 , Accuracy =0.1

 Dist from Wi = 0.553122230840363 , Dist from Wf =18.63634307553735

 n_rows = 1024 , Accuracy =0.1

 Dist from Wi = 0.82385555981543 ,

In [ ]:
np.save("accuracy_UI3.npy", acc)
np.save("distance_i_UI3.npy", dist_i)
np.save("distance_f_UI3.npy", dist_f)